# <div align="center">R-CNN</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)
  
  
<img src="asset/7_2/main.png" />

Object detection is the task of finding the different objects in an image and classifying them (as seen in the image above). The team, comprised of Ross Girshick (a name we’ll see again), Jeff Donahue, and Trevor Darrel found that this problem can be solved with Krizhevsky’s results by testing on the PASCAL VOC Challenge, a popular object detection challenge akin to ImageNet. They write,

Let’s now take a moment to understand how their architecture, Regions With CNNs (R-CNN) works.

The system consists of three modules.
* The first generates category-independent region proposals. These proposals define the set of candidate detections available to our detector. 


* The second module is a large convolutional neural network that extracts a fixed-length feature vector from each region. 


* The third module is a set of classspecific linear SVMs. In this section, we present our design decisions for each module, describe their test-time usage, detail how their parameters are learned, and show detection results on PASCAL VOC 2010-12 and on ILSVRC2013.

# <div align="center">Module Design</div>
---------------------------------------------------------------------

***Region proposals***. A variety of recent papers offer methods for generating category-independent region proposals.

Examples include: objectness, selective search,
category-independent object proposals, constrained
parametric min-cuts (CPMC), multi-scale combinatorial
grouping, and Cires, who detect mitotic cells
by applying a CNN to regularly-spaced square crops, which
are a special case of region proposals. The ***R-CNN uses selective search*** to enable a controlled comparison with prior
detection work.

***Feature extraction***. RCNN extracts a ***4096-dimensional*** feature vector from each region proposal using the Caffe
implementation of the CNN described by Krizhevsky. Features are computed by forward propagating
a mean-subtracted 227 × 227 RGB image through five convolutional layers and two fully connected layers.

<img src='asset/7_2/2.png'>

As the size of each region-proposal is variable and the CNN is designed to take in a 227×227 pixel image, each region-proposal is warped in a tight bounding box, regardless od the size or aspect-ratio, to the required size. In the paper it is mentioned that prior to warping, the bounding box is dilated so that at the warped size there are exactly p pixels of warped image context around the original box (p = 16 is used in the paper). This helps improve the performance.

***SVM Classifiers***.One binary SVM per class is used to classify each region-proposal. The class with the highest score for a region-proposal is assigned to that region. The 4096-dimensional feature vector from the CNN is input to the SVM. In practice,  the feature matrix (instead of a vector) is typically 2000×4096 and the SVM weight matrix is 4096xN where N is the number of classes. Given all scored regions in an image, greedy non-maximum suppression (for each class independently) is applied that rejects a region if it has an intersection-overunion (IoU) overlap with a higher scoring selected region larger than a learned threshold.

***Bounding-Box regressors***. These are basically a linear regression model per class to predict a new detection window given the pool5 features (from the CNN just before the first fully connected layers) for a selective search region proposal. This not required to do detection and just using the SVM would have sufficed. However, the region-proposals are not always completely accurate and this helps correct any defects. Performance was increased after using this.

# <div align="center">Training</div>
---------------------------------------------------------------------

The CNN is pre-trained on the ILSVRC2012 classification dataset (bounding box labels are not available for this data). This is equivalent to using an already trained model such Alex-Net, VGG-19, etc.

Instead of the 1000 ImageNet classes 20 object classes + background are required. Hence, the final fully-connected layer is removed and replaced with a randomly initialised (N + 1)-way classification layer  (where N is the number of object classes, plus 1 for background).

To adapt the CNN to the new task (detection) and the new domain (warped proposal windows), SGD training of the CNN is continued using only warped region-proposals. All region proposals with >= 0.5 IoU overlap with a ground-truth box are considered as positives for the box’s class and the rest as negatives. The ground-truth box here is agnostic to the classes.

To adapt the CNN to the new task (detection) and the new domain (warped proposal windows), SGD training of the CNN is continued using only warped region-proposals. All region proposals with >= 0.5 IoU overlap with a ground-truth box are considered as positives for the box’s class and the rest as negatives. The ground-truth box here is agnostic to the classes.

In each SGD iteration uniformly sample 32 positive windows (over all classes) and 96 background windows to construct a minni-batch of size 128.

A binary SVM is trained for each class.

All region proposals with >= 0.3 IoU overlap with a ground-truth box are considered as positives for the class and the rest as negatives. The IoU threshold used to train the SVMs is different to that used to fine-tune the CNN. Another thing to note is that, in case of fine-tuning the CNN the ground-truth boxes for all the classes were considered but while training an SVM for a particular class only ground-truth box for that particular class is considered.

The final fully-connected layer (introduced during fine-tuning the CNN) is removed and the region-proposals (both positive and negative) are forward propagated to generate a 4096-dimensional feature vector.

Once features are extracted and training labels are applied, we optimise the linear SVM for that class.



Here is the system flow.

<img src='asset/7_2/3.png'>

A simple bounding-box regression stage is used to improve localisation performance. After scoring each selective search proposal with a class-specific detection SVM, a new bounding box is predicted for the detection using a class-specific bounding-box regressor. The input to this are outputs of the Pool5 layer of the CNN. Training is done by minimising the L2 loss.